In [ ]:

#   'Simulated_SWL', 'Simulated_Error_SWL',
#       'Simulated_AQ_THK_2',  
#       'Simulated_V_COND_1', 'Simulated_TRANSMSV_1', 'Simulated_TRANSMSV_2',
#       'er_AQ_THK_2',  'er_H_COND_2','er_AQ_THK_1', 'Simulated_AQ_THK_1', 
#       'er_V_COND_1', 'er_V_COND_2', 'er_TRANSMSV_1', 'er_TRANSMSV_2']


     #   'SWL', 'AQ_THK_2', 'AQ_THK_D',, 'H_COND_2', 'V_COND_1',
     #  'V_COND_2', 'TRANSMSV_1', 'TRANSMSV_2', 'B_AQ_THK', 'B_H_COND',
     #  'B_V_COND', 'B_TRANS', 'AQ_THICK_D', 'H_COND_D', 'V_COND_D', 'TRANS_D',#'H_COND_1',
                 #  'AQ_THK_1',

In [ ]:
###### CLEAN UP PREVIOUS RUN (dont need to be changed)
from ml_function import *
remove_files('*.jpeg', '*.joblib', 'Permutation*.csv','*.h5')
path_to_dataset = '/data/MyDataBase/SWATGenXAppData/observations/rasters_250m_with_observations_without_geometry.pk1'


params = {'font.size': 15,
          'axes.titlesize': 22,
          'axes.labelsize': 22,
          'xtick.labelsize': 18,
          'ytick.labelsize': 18,
          'legend.fontsize': 14,
          'figure.titlesize': 18}

formatted_results = pd.DataFrame(np.zeros((1000,13)), columns=['Model', 'Scenario', 'target', 'avg_nse', 'avg_rmse', 
                                                               'avg_nrmse', 'avg_r2', 'avg_willmott', 'Best_NSE',
                                                               'Best_RMSE', 'Best_nRMSE', 'Best_R2', 'Best_Willmott'])

target_variable = [ 
                    'TRANSMSV_1',
                   ]

##### one example of creating sceanrios.



categorical_features = ['OBJECTID_AQU_CHAR', 'OBJECTID_LDSYSTEM', 'GeologicUnit_GENERALIZE']#, 'flowline', 'FCode', 'FType', 'WB','Soil','landforms' ,'landuse', 'StreamLeve', 'StreamOrde']  ## this is the categorical values that we are interested to incorporate in CNN
num_features = ['Simulated_TRANSMSV_1','er_TRANSMSV_1']#, 'SlopeLenKm', 'Slope',  'MaxElevSmo', 'MinElevSmo']  #

rbf_features=['Elevation']  #'x','y',
########################################### set your scenarios name

scenario_names = [
            'TRANSMSV_1_optimization',
                ]

########################################### CREATE YOU OWN SCENARIOS, exclude parameters, specify categorical features if you need

scenarios = {       
    'TRANSMSV_1_optimization': {'num_features': num_features,
                                'target_variable':target_variable,
                              'rbf_features':rbf_features,
        'exclude_parameter': [],
        'categorical_features': categorical_features
    
    },  
    }


models= [
    
 #    (RandomForestRegressor(random_state=42), {'model__n_estimators': [50, 100, 200], 
 #                                             'model__max_depth': [None, 2, 5], 
 #                                             'model__min_samples_split': [4, 8],
 #                              'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(),
 #                              PowerTransformer()],
 #                                             'model__min_samples_leaf': [1, 2], 
 #                                             'model__max_features': ['sqrt'],
 #                                             'model__bootstrap': [True]}, "random_forest","Random"),
    
        (XGBRegressor(random_state=42, tree_method='hist', booster='dart'),
     {
   #   'model__booster': optuna.distributions.CategoricalDistribution(['gbtree', 'gblinear', 'dart']),
       'model__booster': optuna.distributions.CategoricalDistribution(['dart']),

      'model__rate_drop': optuna.distributions.FloatDistribution(0, 0.2),  # DART-specific: rate_drop
      'model__one_drop': optuna.distributions.CategoricalDistribution([0, 1]),  # DART-specific: one_drop
         
    # 'model__sample_type': optuna.distributions.CategoricalDistribution(['uniform', 'weighted']),  # DART-specific: sample_type
      'model__sample_type': optuna.distributions.CategoricalDistribution(['weighted']),  # DART-specific: sample_type

      'model__normalize_type': optuna.distributions.CategoricalDistribution(['forest']),  # DART-specific: normalize_type
         
      'model__n_estimators': optuna.distributions.IntDistribution(30, 50, step=10),
      'model__max_depth': optuna.distributions.IntDistribution(2, 6),
      'model__min_child_weight': optuna.distributions.IntDistribution(4, 8),
         
   #   'model__colsample_bytree': optuna.distributions.CategoricalDistribution([0.5, 0.6, 0.7]),
      'model__subsample': optuna.distributions.FloatDistribution(0.6, 0.9),
         
      'model__learning_rate': optuna.distributions.FloatDistribution(0.01, 0.1),
   #   'model__gamma': optuna.distributions.FloatDistribution(0, 2),
         
   #   'model__reg_alpha': optuna.distributions.FloatDistribution(0.1, 0.5),
   #   'model__reg_lambda': optuna.distributions.FloatDistribution(0.1, 0.5),
         
  #    'model__scale_pos_weight': optuna.distributions.FloatDistribution(1, 5)
     },
     "gradient_boosting_gpu", 'Optuna') ,
    
    
    
    
        (XGBRFRegressor(random_state=42, tree_method='hist'), 
     {'model__n_estimators': optuna.distributions.IntDistribution(50, 250, step=50),
      'model__max_depth': optuna.distributions.IntDistribution(2, 10),
      'model__min_child_weight': optuna.distributions.IntDistribution(1,10),
       'model__colsample_bynode': optuna.distributions.CategoricalDistribution([0.5])}, 
     "random_forest_gpu", 'Optuna'),
    

]

######## MAKING STACKED MODELS? if yes, Turn STACK_MODELS==True
models = stacking_models (models, optimization='Optuna',STACK_MODELS=True)


### costum scorer will be used in grid search (hyper parameter optimization) and permutation analysis
### for outter  
DDDD=0
number_of_samples=500000
County_specific=None
# Nested cross-validation configuation
inner_cv = KFold(n_splits=5, shuffle=True, random_state=42)

print("Scenario names: ",scenario_names)

for scenario_name in scenario_names:
    
    print(" ############## Scenario name:",scenario_name,' ############## ')
        
    num_features =scenarios[scenario_name]['num_features']
    exclude_parameter=scenarios[scenario_name]['exclude_parameter']
    target_variable=scenarios[scenario_name]['target_variable']
    categorical_features=scenarios[scenario_name]['categorical_features']
    rbf_features=scenarios[scenario_name]['rbf_features']
    
    print('num_features length:',len(num_features))
    print('rbf_features length:', len(rbf_features))
    print('categorical_features length:', len(categorical_features))
    
    print('Categorical features:',categorical_features)
    
    custome_scorer = make_scorer(kling_gupta_efficiency, greater_is_better=True)                                       ##### step1: define scorer for inner loop

    for model, param_grid, model_name, optimization_method in models:
        
        print(' ############## Model being used for prediction:',model_name,' ############## ')
        
        input_dataset = import_dataset(path_to_dataset,number_of_samples, COUNTY=County_specific, sampling_group=['TOWNSHIP'],grid=False)                                                ##### import dataset, COUNTY: for working on one or several counties instead of entire data
        print('number of Counties in the dataset:',len(input_dataset.COUNTY.unique()))

        
        print('############## target parameter for prediction:',target_variable,' ############## ')
        
        input_dataset = filter_by_quantiles(input_dataset, target_variable, lower_quantile=0, upper_quantile=1)           #### filter importer dataset based on quantile

        input_dataset[categorical_features] = input_dataset[categorical_features].astype('category')                           #### define categorical feature data type
        input_dataset[rbf_features] = input_dataset[rbf_features].astype(int)                             #### define categorical feature data type

        output_variable = input_dataset.loc[:, target_variable[0]]                                                    #### define target feature

        print('$$$$$ PIPLINE: Ensamble model $$$$$')
        input_variables = input_dataset.loc[0:,num_features+categorical_features+rbf_features+['COUNTY']]                            ####  define input features (both categorical and numerical) 
        groups = input_variables['COUNTY'].values  # note: we have 68 counties
        group_split = int(len(np.unique(list(groups)))/17)
        group_kfold = GroupKFold(n_splits=group_split)
        input_variables = input_variables.drop(columns='COUNTY')
        outer_cv = GroupKFold(n_splits=group_split).split(input_variables, output_variable, groups)

        print('numerical features:',num_features)                                                                 
        print_statistics(output_variable,target_variable[0])                                                           #### print target feature stats

        rbf_processor = RBFSampler(gamma=1, random_state=42)

        preprocessor = ColumnTransformer(
            transformers=[
            ('num', 'passthrough', num_features),
            ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features),

                ('rbf', rbf_processor, rbf_features)
        ])

        pipeline = Pipeline([ 

            ('preprocessor', preprocessor),
            ('scaler', 'passthrough'),  # Placeholder for scaler
            ('model', model)
        ])

        if optimization_method=='Random':
            optimization_search = RandomizedSearchCV(pipeline, param_grid, scoring=custome_scorer, cv=inner_cv, n_jobs=60, verbose=0)                                    #### randomize search                                   
        elif optimization_method=='Optuna':
            optimization_search = OptunaSearchCV(estimator=pipeline,param_distributions=param_grid,scoring=custome_scorer,n_trials=25, cv=inner_cv ,n_jobs=60,verbose=0)   ##### Optuna Search
        elif optimization_method=='Grid':
            optimization_search = GridSearchCV(pipeline, param_grid, scoring=custome_scorer, cv=inner_cv, n_jobs=60, verbose=0)                                          ##### Grid Search


        print(f" ############## optimization method: {optimization_method} ############## ")
        
        best_score, best_model, nse_scores, r2_scores, rmse_scores, nrmse_scores, willmott_scores = initialize_metrics()                                      ## initialize
        
        avg_nse, avg_r2, avg_rmse, avg_nrmse, avg_willmott, best_model         = nested_cross_validation(outer_cv, input_variables, output_variable, optimization_search, nse_function, rmse_function, willmott_index, groups,group_kfold)   ## Perform nested cross validation 
       
        best_y_pred, best_wi_score, best_nse, best_rmse, best_nrmse, best_r2   = test_best_model(model_name, scenario_name, target_variable[0], input_variables, output_variable, best_model, willmott_index)  ## save the best model found in nested cross validation and then load it and tested and get the performance.

        plot_results(output_variable, target_variable[0], best_y_pred, avg_rmse, avg_nrmse, avg_nse, avg_r2, avg_willmott, best_rmse, best_nrmse, best_nse, best_r2, best_wi_score, model_name, scenario_name, DDDD,params)  ## plot the results and present the best performance and the average of nested cross validation for the outer layer
        
        formatted_results = summarize_results(formatted_results, DDDD, model_name, scenario_name, target_variable,best_rmse, best_nrmse, best_nse, best_r2, best_wi_score,avg_nse,avg_rmse,avg_nrmse,avg_r2,avg_willmott)   ## summarize the results
        
        model_name_label=' '.join(word.capitalize() for word in model_name.split('_'))
        permutation_importance_analysis(optimization_search.best_estimator_, input_variables, output_variable,model_name_label,scenario_name,target_variable[0],custome_scorer)  ## perform permutation analysis
        DDDD=DDDD+1

formatted_results[formatted_results.avg_rmse>0].to_csv('Scenarios_all_models_bagging.csv')

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from ml_function import *

categorical_features = [ 'Soil','landuse',
                        'StreamLeve', 
                        'StreamOrde',
                        'AQU_CHAR', 
                        'LDSYSTEM', 
                        'GeologicUnit_GENERALIZE',
                        'flowline',
                        'FCode', 
                        'FType', 
                         'WB'
                       ]

num_features=['er_TRANSMSV_1',
            'Simulated_TRANSMSV_1',
            'SlopeLenKm',
            'Slope',
            'MaxElevSmo',
            'MinElevSmo']

rbf_features=['x','y','Top_Elev']
def plot_county_point(dataset, COUNTY, col_sim, col_ml, col_obs):
    dataset = dataset[~dataset.COUNTY.isna()]
   # dataset = dataset[~dataset.col_sim.isna()]

    if 'geometry' not in dataset.columns:
        X = dataset.x.values
        Y = dataset.y.values
        points = [Point(x, y) for x, y in zip(X, Y)]
        dataset.loc[:,'geometry'] = points
        dataset = gpd.GeoDataFrame(dataset, geometry='geometry', crs='EPSG:26990').reset_index(drop=True)

    fig, axes = plt.subplots(1, 3, figsize=(50, 30))
    
    dataset.plot(column=col_sim, s=1, cmap='Spectral_r', ax=axes[0])
    axes[0].set_title('Krigging')
    axes[0].set_axis_off()
    dataset.plot(column=col_ml, s=1, cmap='Spectral_r', ax=axes[1])
    axes[1].set_title('Krigging Error')
    axes[1].set_axis_off()
    dataset.plot(column=col_obs, s=1, cmap='Spectral_r', ax=axes[2])
    axes[2].set_title('Machine learning')
    axes[2].set_axis_off()
    plt.savefig(f'_{uuid.uuid4()}_{COUNTY}.jpeg', dpi=300)
    plt.show()

    return dataset

COUNTY=['Huron','Tuscola','Sanilac']

dataset = pd.read_pickle('grid_points_well_obs.pk1')
#best_rf_model = joblib.load('best_rf_model_gradient_boosting_gpu_TRANSMSV_1_optimization_TRANSMSV_1.joblib')
best_rf_model = joblib.load('best_rf_model_random_forest_gpu_TRANSMSV_1_optimization_TRANSMSV_1.joblib')
y_pred = best_rf_model.predict(dataset.loc[:,num_features+categorical_features+rbf_features] )
dataset['ML']=y_pred
dataset = plot_county_point(dataset, COUNTY, 'Simulated_TRANSMSV_1', 'er_TRANSMSV_1','ML')

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

def plot_county_point(dataset, COUNTY, col_sim, col_ml, col_obs):
    #dataset = dataset[dataset.COUNTY == COUNTY]
    dataset = dataset[~dataset.COUNTY.isna()]
    if 'geometry' not in dataset.columns:
        X = dataset.x.values
        Y = dataset.y.values
        points = [Point(x, y) for x, y in zip(X, Y)]
        dataset['geometry'] = points
        dataset = gpd.GeoDataFrame(dataset, geometry='geometry', crs='EPSG:26990').reset_index(drop=True)

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Define a common color range for all plots
    vmin = dataset[[col_sim, col_ml, col_obs]].min().min()
    vmax = dataset[[col_sim, col_ml, col_obs]].max().max()
    
    # Plot with the same color range and add color bars
    plot_sim = dataset.plot(column=col_sim, s=1, cmap='Spectral_r', vmin=vmin, vmax=vmax, ax=axes[0])
    axes[0].set_title('Krigging')
    axes[0].set_axis_off()
    plt.colorbar(plot_sim.get_children()[0], ax=axes[0], orientation='vertical')

    plot_ml = dataset.plot(column=col_ml, s=1, cmap='Spectral_r', vmin=vmin, vmax=vmax, ax=axes[1])
    axes[1].set_title('Krigging Error')
    axes[1].set_axis_off()
    plt.colorbar(plot_ml.get_children()[0], ax=axes[1], orientation='vertical')

    plot_obs = dataset.plot(column=col_obs, s=1, cmap='Spectral_r', vmin=vmin, vmax=vmax, ax=axes[2])
    axes[2].set_title('Machine learning')
    axes[2].set_axis_off()
    plt.colorbar(plot_obs.get_children()[0], ax=axes[2], orientation='vertical')
    plt.tight_layout()
    plt.savefig(f'_{uuid.uuid4()}_{COUNTY}.jpeg', dpi=300)
    plt.show()

    return dataset



dataset = pd.read_pickle('grid_points_well_obs.pk1')
#best_rf_model = joblib.load('best_rf_model_gradient_boosting_gpu_TRANSMSV_1_optimization_TRANSMSV_1.joblib')
best_rf_model = joblib.load('best_rf_model_random_forest_gpu_TRANSMSV_1_optimization_TRANSMSV_1.joblib')

y_pred = best_rf_model.predict(dataset.loc[:,num_features+categorical_features+rbf_features] )

dataset['ML']=y_pred


dataset = plot_county_point(dataset, COUNTY, 'Simulated_TRANSMSV_1', 'er_TRANSMSV_1','ML')


